In [33]:
from ts2vec import TS2Vec
from data_provider import data_loader
from thresholding import *
from eval import *
from tools import *
from novelty import *
from simple_baselines import *
import torch
import numpy as np
import os
import time
import datetime

In [34]:
#PSM = data_loader.PSMSegLoader('../TAD-Analysis/dataset/PSM', 100)
#SMD = data_loader.SMDSegLoader('../TAD-Analysis/dataset/SMD', 100)
#MSL = data_loader.MSLSegLoader('../TAD-Analysis/dataset/MSL', 100)
#SWAT = data_loader.SWATSegLoader('../TAD-Analysis/dataset/SWaT', 100)
#WADI = data_loader.WADISegLoader('../TAD-Analysis/dataset/WADI', 100)
#SMAP = data_loader.SMAPSegLoader('../TAD-Analysis/dataset/SMAP', 100)
#SWAN = data_loader.NIPS_TS_SwanSegLoader('../TAD-Analysis/dataset/NIPS_TS_Swan', 100)
GECCO = data_loader.NIPS_TS_SwanSegLoader('../TAD-Analysis/dataset/NIPS_TS_Water', 100)

train: (60000, 38)
test: (60000, 38)


In [28]:
train_data, vali_data, test_data, test_labels = obtain_data(PSM)
test_labels = PSM.test_labels.reshape(-1)
del PSM

In [3]:
train_data, vali_data, test_data, test_labels = obtain_data(SMD)
del SMD

In [3]:
train_data, vali_data, test_data, test_labels = obtain_data(MSL)
del MSL

In [3]:
train_data, vali_data, test_data, test_labels = obtain_data(SWAT)
test_labels = test_labels.reshape(-1)
del SWAT

In [3]:
train_data, vali_data, test_data, test_labels = obtain_data(WADI)
test_labels = test_labels.reshape(-1)
del WADI

In [3]:
train_data, vali_data, test_data, test_labels = obtain_data(SMAP)
del SMAP

In [35]:
train_data, vali_data, test_data, test_labels = obtain_data(SWAN)
del SWAN

In [3]:
train_data, vali_data, test_data, test_labels = obtain_data(GECCO)
del GECCO

In [36]:
train_data = train_data.reshape(1, train_data.shape[0], train_data.shape[1])
vali_data = vali_data.reshape(1, vali_data.shape[0], vali_data.shape[1])
test_data = test_data.reshape(1, test_data.shape[0], test_data.shape[1])

In [37]:
model = TS2Vec(input_dims=train_data.shape[-1])

In [38]:
#model.load('training/PSM/model.pkl')
#model.load('training/SMD/model.pkl')
#model.load('training/MSL/model.pkl')
#model.load('training/SWAT/model.pkl')
#model.load('training/WADI/model.pkl')
#model.load('training/SMAP/model.pkl')
model.load('training/SWAN/model.pkl')

In [39]:
#model.save('training/PSM/model.pth')
#model.save('training/SMD/model.pth')
#model.save('training/MSL/model.pth')
#model.save('training/SWAT/model.pth')
#model.save('training/WADI/model.pth')
model.save('training/SWAN/model.pth')

In [6]:
t = time.time()
loss_log = model.fit(train_data, verbose=True)

Epoch #0: loss=44.53508949279785
Epoch #1: loss=9.87163782119751
Epoch #2: loss=11.581430196762085
Epoch #3: loss=3.4308485984802246
Epoch #4: loss=4.922553539276123
Epoch #5: loss=4.251410722732544
Epoch #6: loss=16.683390855789185
Epoch #7: loss=4.29343318939209
Epoch #8: loss=14.681182384490967
Epoch #9: loss=9.664965152740479
Epoch #10: loss=5.421787977218628
Epoch #11: loss=6.821531176567078
Epoch #12: loss=4.5636667013168335
Epoch #13: loss=5.0421226024627686
Epoch #14: loss=5.104500770568848
Epoch #15: loss=3.4345680475234985
Epoch #16: loss=4.092029333114624
Epoch #17: loss=3.214359760284424
Epoch #18: loss=3.7743834257125854
Epoch #19: loss=3.4874281883239746
Epoch #20: loss=3.1366814374923706
Epoch #21: loss=3.9748822450637817
Epoch #22: loss=3.2060697078704834
Epoch #23: loss=3.387554883956909
Epoch #24: loss=2.9225815534591675
Epoch #25: loss=2.656099557876587
Epoch #26: loss=2.915250301361084
Epoch #27: loss=2.8125563859939575
Epoch #28: loss=2.795840263366699
Epoch #29: l

In [7]:
t = time.time() - t
print(f"\nTraining time: {datetime.timedelta(seconds=t)}\n")


Training time: 0:02:19.827754



In [10]:
plot_loss(loss_log)

In [8]:
# save the trained model
dataset = 'GECCO'
run_dir = 'training/' + dataset
os.makedirs(run_dir, exist_ok=True)
model.save(f'{run_dir}/model.pkl')

In [9]:
# anomaly detection evaluation
mask_test_repr = model.encode(
    test_data,
    mask='mask_last',
    causal=True,
    sliding_length=1,
    sliding_padding=100,
    batch_size=128
).squeeze()

In [10]:
wo_mask_test_repr = model.encode(
    test_data,
    causal=True,
    sliding_length=1,
    sliding_padding=100,
    batch_size=128
).squeeze()

In [11]:
test_scores = []
for k in range(mask_test_repr.shape[0]):
    test_score = np.abs(wo_mask_test_repr[k] - mask_test_repr[k]).sum(axis=0)
    test_scores.append(test_score)
test_scores = np.array(test_scores)

In [12]:
print("ts2vec AD ROC-AUC:", calculate_roc_auc(test_labels, test_scores))
print("ts2vec AD PR-AUC:", calculate_pr_auc(test_labels, test_scores))

ts2vec AD ROC-AUC: 0.7531770343708069
ts2vec AD PR-AUC: 0.27467783974405896


In [13]:
# Compute timestamp-level representations
time_train_repr = model.encode(train_data)
time_vali_repr = model.encode(vali_data)
time_test_repr = model.encode(test_data)

In [14]:
time_train_repr = time_train_repr.reshape(-1, 320)
time_vali_repr = time_vali_repr.reshape(-1, 320)
time_test_repr = time_test_repr.reshape(-1, 320)

In [15]:
ts2_mean_scores = np.mean(time_test_repr, axis=1)
print("ts2vec Mean ROC-AUC:", calculate_roc_auc(test_labels, ts2_mean_scores))
print("ts2vec Mean PR-AUC:", calculate_pr_auc(test_labels, ts2_mean_scores))

ts2vec Mean ROC-AUC: 0.316647640513852
ts2vec Mean PR-AUC: 0.006978219042352669


In [16]:
ts2_max_scores = np.max(time_test_repr, axis=1)
print("ts2vec Max ROC-AUC:", calculate_roc_auc(test_labels, ts2_max_scores))
print("ts2vec Max PR-AUC:", calculate_pr_auc(test_labels, ts2_max_scores))

ts2vec Max ROC-AUC: 0.3662241645266826
ts2vec Max PR-AUC: 0.05110379455474393


In [17]:
print("IF training based on mixed data")
combined = np.concatenate((time_train_repr, time_vali_repr), axis=0)
ts2_if_scores = fit_isolation_forest(combined, time_test_repr)
print("ts2vec IF ROC-AUC:", calculate_roc_auc(test_labels, ts2_if_scores))
print("ts2vec IF PR-AUC:", calculate_pr_auc(test_labels, ts2_if_scores))

IF training based on mixed data
ts2vec IF ROC-AUC: 0.6807833391267986
ts2vec IF PR-AUC: 0.24115059453494014


In [18]:
print("IF training based on only training data")
ts2_if_scores = fit_isolation_forest(time_train_repr, time_test_repr)
print("ts2vec IF ROC-AUC:", calculate_roc_auc(test_labels, ts2_if_scores))
print("ts2vec IF PR-AUC:", calculate_pr_auc(test_labels, ts2_if_scores))

IF training based on only training data
ts2vec IF ROC-AUC: 0.6646339532774189
ts2vec IF PR-AUC: 0.252667942897619


In [19]:
print("IF training based on only validation data")
ts2_if_scores = fit_isolation_forest(time_vali_repr, time_test_repr)
print("ts2vec IF ROC-AUC:", calculate_roc_auc(test_labels, ts2_if_scores))
print("ts2vec IF PR-AUC:", calculate_pr_auc(test_labels, ts2_if_scores))

IF training based on only validation data
ts2vec IF ROC-AUC: 0.7328607411544382
ts2vec IF PR-AUC: 0.2568556065143856


In [20]:
train_data = train_data.reshape(-1, train_data.shape[2])
vali_data = vali_data.reshape(-1, vali_data.shape[2])
test_data = test_data.reshape(-1, test_data.shape[2])

In [21]:
norm_baseline = LNorm() 
norm_baseline.fit(train_data)
norm_scores = norm_baseline.transform(test_data)
print("Norm ROC-AUC:", calculate_roc_auc(test_labels, norm_scores))
print("Norm PR-AUC:", calculate_pr_auc(test_labels, norm_scores))

Norm ROC-AUC: 0.7351054507279278
Norm PR-AUC: 0.2770556555848389


In [22]:
pca_baseline = PCAError()
pca_baseline.fit(train_data)
pca_vali = pca_baseline.transform(vali_data)
pca_test = pca_baseline.transform(test_data)
pca_max_scores = np.max(pca_test, axis=1)
pca_mean_scores = np.mean(pca_test, axis=1)
pca_if_scores = fit_isolation_forest(pca_vali, pca_test)
print("PCA Mean ROC-AUC:", calculate_roc_auc(test_labels, pca_mean_scores))
print("PCA Mean PR-AUC:", calculate_pr_auc(test_labels, pca_mean_scores))
print("PCA Max ROC-AUC:", calculate_roc_auc(test_labels, pca_max_scores))
print("PCA Max PR-AUC:", calculate_pr_auc(test_labels, pca_max_scores))
print("PCA IF ROC-AUC:", calculate_roc_auc(test_labels, pca_if_scores))
print("PCA IF PR-AUC:", calculate_pr_auc(test_labels, pca_if_scores))

Adjusting estimated number of PCA components from 10 to 2.
PCA Mean ROC-AUC: 0.7838885331965555
PCA Mean PR-AUC: 0.27189430522466673
PCA Max ROC-AUC: 0.6200358981626753
PCA Max PR-AUC: 0.22379140496165104
PCA IF ROC-AUC: 0.8286023043666071
PCA IF PR-AUC: 0.26218131379039766


In [23]:
pca_baseline = PCAError()
pca_baseline.fit(time_train_repr)
pca_vali = pca_baseline.transform(time_vali_repr)
pca_test = pca_baseline.transform(time_test_repr)
pca_max_scores = np.max(pca_test, axis=1)
pca_mean_scores = np.mean(pca_test, axis=1)
pca_if_scores = fit_isolation_forest(pca_vali, pca_test)
print("PCA Mean ROC-AUC:", calculate_roc_auc(test_labels, pca_mean_scores))
print("PCA Mean PR-AUC:", calculate_pr_auc(test_labels, pca_mean_scores))
print("PCA Max ROC-AUC:", calculate_roc_auc(test_labels, pca_max_scores))
print("PCA Max PR-AUC:", calculate_pr_auc(test_labels, pca_max_scores))
print("PCA IF ROC-AUC:", calculate_roc_auc(test_labels, pca_if_scores))
print("PCA IF PR-AUC:", calculate_pr_auc(test_labels, pca_if_scores))

PCA Mean ROC-AUC: 0.7936450917223142
PCA Mean PR-AUC: 0.12597176805790605
PCA Max ROC-AUC: 0.8114716607602639
PCA Max PR-AUC: 0.11555371840139511
PCA IF ROC-AUC: 0.7777925318469013
PCA IF PR-AUC: 0.11991858969362504


In [24]:
random_baseline = Random()
random_baseline.fit(train_data)
random_scores = random_baseline.transform(test_data)
print("Random ROC-AUC:", calculate_roc_auc(test_labels, random_scores))
print("Random PR-AUC:", calculate_pr_auc(test_labels, random_scores))

Random ROC-AUC: 0.5009503448394418
Random PR-AUC: 0.2592800942354089


In [25]:
raw_mean_scores = np.mean(test_data, axis=1)
raw_max_scores = np.max(test_data, axis=1)
raw_if_scores = fit_isolation_forest(vali_data, test_data)
print("Raw Mean ROC-AUC:", calculate_roc_auc(test_labels, raw_mean_scores))
print("Raw Mean PR-AUC:", calculate_pr_auc(test_labels, raw_mean_scores))
print("Raw Max ROC-AUC:", calculate_roc_auc(test_labels, raw_max_scores))
print("Raw Max PR-AUC:", calculate_pr_auc(test_labels, raw_max_scores))
print("Raw IF ROC-AUC:", calculate_roc_auc(test_labels, raw_if_scores))
print("Raw if PR-AUC:", calculate_pr_auc(test_labels, raw_if_scores))

Raw Mean ROC-AUC: 0.29354847511265275
Raw Mean PR-AUC: 0.07573969348871169
Raw Max ROC-AUC: 0.2713879909961755
Raw Max PR-AUC: 0.04356793442587402
Raw IF ROC-AUC: 0.7576722802979073
Raw if PR-AUC: 0.17095564400733537


In [31]:
raw_if_scores = fit_isolation_forest(vali_data, test_data)
opt_th = best_f_score(raw_if_scores, test_labels)
calculate_metrics(test_labels, raw_if_scores, opt_th)

F-Score: 0.48271273544864296
F-PA Score: 0.538475138121547
F-PAK Score: 0.538475138121547
F-K-AUC Score: 0.5247518329491738
Composite F-score: 0.49075331609420236


In [32]:
raw_if_scores = fit_isolation_forest(train_data, test_data)
opt_th = best_f_score(raw_if_scores, test_labels)
calculate_metrics(test_labels, raw_if_scores, opt_th)

F-Score: 0.519905591614314
F-PA Score: 0.7270936254682766
F-PAK Score: 0.6925341100720527
F-K-AUC Score: 0.5968245983048163
Composite F-score: 0.5916347872065948


In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

data = {
    'Model': ['PCA', 'ts2vec', 'Recons', 'Untrecons', 'L2-norm', 'Random', 'Raw'],
    'PR-AUC': [0.6138, 0.5669, 0.5424, 0.5348, 0.4728, 0.4569, 0.4286],
    'ROC-AUC': [0.7846, 0.7226, 0.7884, 0.7495, 0.6584, 0.4996, 0.7240]
}

df = pd.DataFrame(data)
sns.set(style="whitegrid")
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

sns.barplot(x='PR-AUC', y='Model', data=df, ax=ax[0], palette='viridis')
ax[0].set_title('PSM PR-AUC Scores')
ax[0].set_xlabel('PR-AUC Score')
ax[0].set_ylabel('Model')

sns.barplot(x='ROC-AUC', y='Model', data=df, ax=ax[1], palette='viridis')
ax[1].set_title('PSM ROC-AUC Scores')
ax[1].set_xlabel('ROC-AUC Score')
ax[1].set_ylabel('Model')

ax[0].set_xlim(0, 0.8)
ax[1].set_xlim(0, 0.8)
plt.tight_layout()
file_name = 'PSM_comparison.png'
plt.savefig(file_name)
plt.show()